In [ ]:
!pip install --upgrade langchain langchain-community  langchain-groq wikipedia
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.2 MB/s eta 0:00:00


In [ ]:
from langchain_community.tools import WikipediaQueryRun #we can ask question to wikipedia
from langchain_community.utilities import WikipediaAPIWrapper # we can fetch answer from wikipedia

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max =1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper) # takes user query and search the wikipedia and show the top 1 result

In [ ]:
wiki_tool.run({"query": "AI agents"})

'Page: Agentic AI\nSummary: Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence.'

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=1, api_key ="--", model = "llama-3.1-8b-instant")

In [ ]:
tools = [wiki_tool]

#Tool binding
llm_with_tools = llm.bind_tools(tools)

#Tool calling
result = llm_with_tools.invoke("Hello bro!")
result
result.content

"What's up? How can I help you today? Do you want to know something?"

## Creating Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools)

In [ ]:
from langchain_core.messages import HumanMessage

# It respond when there's no need to call a tool:

response = agent_executor.invoke({"messages": [HumanMessage(content="Hello bro!")]})
response["messages"]

[HumanMessage(content='Hello bro!', additional_kwargs={}, response_metadata={}, id='8efe5fa6-773f-4542-8ddf-928a2caeb643'),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 284, 'total_tokens': 292, 'completion_time': 0.013153797, 'prompt_time': 0.022221409, 'queue_time': 0.21342996, 'total_time': 0.035375206}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'finish_reason': 'stop', 'logprobs': None}, id='run--d379484a-8c04-470e-a1b0-5fb35fe7e5ed-0', usage_metadata={'input_tokens': 284, 'output_tokens': 8, 'total_tokens': 292})]

In [ ]:
print(response["messages"][-1].content)

I'm happy to chat with you. What's on your mind?


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="What is agentic ai?")]})

# Extract only the relevant answer parts from ToolMessage and AIMessage
clean_output = []
for message in response["messages"]:
    if hasattr(message, 'content') and message.content:
        if isinstance(message, ToolMessage):
            clean_output.append(message.content)
        elif isinstance(message, AIMessage) and message.content.strip():
            clean_output.append(message.content)

# Print the clean final answer
print("\n".join(clean_output))

Page: Agentic AI
Summary: Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence.
The provided summary is based on the Wikipedia page for Agentic AI.


## Intractive mode

# Workflow
[User Input] ➡️ [LLM processes message] ➡️ [Tool call to Wikipedia] ➡️ [ToolMessage with result] ➡️ [LLM creates AIMessage reply] ➡️ [Shown to user]


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import threading
import time

def loading_animation(stop_event):
    while not stop_event.is_set():
        for ch in "|/-\\":
            print(f"\rAssistant is thinking... {ch}", end="", flush=True)
            time.sleep(0.2)

chat_history = []

while True:
    user_input = input("\nUser: ")

    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    chat_history.append(HumanMessage(content=user_input))

    stop_event = threading.Event()
    loader_thread = threading.Thread(target=loading_animation, args=(stop_event,))
    loader_thread.start()

    try:
        response = agent_executor.invoke({"messages": chat_history})

        stop_event.set()
        loader_thread.join()
        print("\r" + " " * 50 + "\r", end="")  # Clear the loading line

        # Store already shown pages to avoid duplicates
        shown_pages = set()

        # Show ToolMessage (Wikipedia result)
        for message in response["messages"]:
            if isinstance(message, ToolMessage):
                page_title = message.content.split("\n")[0]
                if page_title not in shown_pages:
                    shown_pages.add(page_title)
                    print(message.content)

        # Show the final AIMessage answer
        final_answer = None
        for message in reversed(response["messages"]):
            if isinstance(message, AIMessage) and message.content.strip():
                final_answer = message.content
                chat_history.append(message)
                break

        if final_answer:
            print(final_answer)
        else:
            print("No answer found.")

    except Exception as e:
        stop_event.set()
        loader_thread.join()
        print("\r" + " " * 50 + "\r", end="")  # Clear the loading line
        print(f"Error occurred: {e}")



User: hi
I'm happy to chat with you. How can I assist you today?

User: what is agent ai?
Page: Agentic AI
Summary: Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence.
Note that my knowledge cutoff is December 2023, so the information I provide may not be up to date or comprehensive.

User: so, tell me what is llm?
Page: Large language model
Summary: A large language model (LLM) is a language model trained with self-supervised machine learning on a vast amount of text, designed for natural language processing tasks, especially language generation.
The largest